In [1]:
! pip install azure-cosmos


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
! pip install json
! pip install python-dotenv

! pip install openai

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
# Import the required libraries
import time
import os
import json
import uuid
from dotenv import dotenv_values
from openai import OpenAI, AzureOpenAI


#Cosmos DB imports
from azure.cosmos import CosmosClient

In [79]:
env_name = "variable.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

cosmos_conn = config['NOSQL_PRIMARY_CONNECTION_STRING']
cosmos_key = config['NOSQL_PRIMARY_KEY']
cosmos_database = "Account"
cosmos_container = "Transactions"
nosql_uri=config["NOSQL_URI"]

openai_endpoint = config['AOAI_ENDPOINT']
openai_key = config['AOAI_KEY']
openai_api_version = config['API_VERSION']
openai_embeddings_deployment = config['AOAI_EMBEDDING_DEPLOYMENT']
openai_embeddings_model = config['AOAI_EMBEDDING_DEPLOYMENT_MODEL']
#openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])

# Create the Azure Cosmos DB for NoSQL client
cosmos_client = CosmosClient(url=nosql_uri, credential=cosmos_key)

os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
client = OpenAI()

azure_openai_embeddings = AzureOpenAI(
    api_version=openai_api_version,
    api_key= openai_key,
    azure_endpoint= openai_endpoint,
)

In [80]:
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = azure_openai_embeddings.embeddings.create(input=text, model=openai_embeddings_model)
    embeddings =response.model_dump()
    time.sleep(0.5) 
    return embeddings['data'][0]['embedding']

In [64]:
# Load text-sample_w_embeddings.json which has embeddings pre-computed
data_file = open(file="test.json", mode="r") 

# OR Load text-sample.json data file. Embeddings will need to be generated using the function below.
# data_file = open(file="../../DataSet/AzureServices/text-sample.json", mode="r")

data = json.load(data_file)
data_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'test.json'

In [66]:
# Take a peek at one data item
print(json.dumps(data[2], indent=2))

{
  "_id": {
    "$oid": "66e75576b5224a4bb5787432"
  },
  "TransactionID": "T7356",
  "Amount": 360.73,
  "Timestamp": "2024-09-15 14:04:38",
  "Location": "New York",
  "Merchant": "Amazon",
  "Fraud": false
}


In [81]:
n = 0
for item in data:
    n+=1
    item['TransactionID'] = str(n)
    amount = item["Amount"]
    timestamp = item["Timestamp"]
    location = item["Location"]
    merchant = item["Merchant"]
    location = get_city_coordinates(location)
    location_embeddings = generate_embeddings(location)
    item['locationVector'] = location_embeddings
    item['@search.action'] = 'upload'
    print("Creating embeddings for item:", n, "/" ,len(data), end='\r')

The latitude of Boston is: 42.3554334
The longitude of Boston is: -71.060511


NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

In [41]:
! pip install geopy

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ---------------------- ---------------- 71.7/125.4 kB 787.7 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [49]:
from geopy.geocoders import Nominatim

def get_city_coordinates(city_name):
    try:
        # Create a geolocator object using Nominatim service
        geolocator = Nominatim(user_agent="MyAPP")
        
        # Geocode the city name to get location details
        location = geolocator.geocode(city_name)
        
        if location:
            # Extract the latitude and longitude from the location object
            lat = location.latitude
            lon = location.longitude
            print(f"The latitude of {city_name} is: {lat}")
            print(f"The longitude of {city_name} is: {lon}")
            return lat, lon
        else:
            print(f"City '{city_name}' not found.")
            return None, None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None, None

